# SBstoat Tutorial

This tutorial discusses helper codes you can use for model fitting with tellurium. The tutorial gives practical examples of to use these codes in a model fitting workflow.

For those who want to get right to the quick start section, the follwing information can be scipped over and is only required for advanced users

The utilities are in 3 python modules (files):

- named_timeseries defines a NamedTimeseries type that allows you to manipulate tabular data that has a time axis. A NamedTimeseries can be initialized from a comma separated variable (CSV) file.
- model_fitter defines a ModelFitter type that encapsulates the basics of fitting parameters to a NamedTimeseries. You can simulate the fitted model, and the result is a NamedTimeseries.
- timeseries_plotter plots one or two NamedTimeseries. You can have all variables on a single subplot or have separate subplots for each variable.

You should open this notebook from ``SBstoat``, then open the folder ``notebooks`` to find this notebook.

## Quick Start

We need to first import the SBstoat python package. You can either use the 'from' symtax shown below or the simpler 'import' syntax, eg import SBstoat
The simpler syntax might be more convenient if you want to use the other functions in the SBstoat package because you can use the console help system
to quickly identifiy the different packages in SBstoat. Just remember,  when using the simpler syntax use the prefix 'modelFitter.ModeFitter'
when creating the fitter object. Note that 'modelFitter' is the package name and 'ModelFitter' with a capital M, is the function that will create the fitter object.


In [1]:
# Basic imports
import SBstoat
%matplotlib inline

Let's first define our model. This is a five-step pathway with five unknown parameterss, k1 to k6



In [2]:
# Model of a linear pathway
ANTIMONY_MODEL = """ 
# Reactions   
    J1: S1 -> S2; k1*S1
    J2: S2 -> S3; k2*S2
    J3: S3 -> S4; k3*S3
    J4: S4 -> S5; k4*S4
    J5: S5 -> S6; k5*S5;
# Species initializations
    S1 = 10;
    S2 = 0;
    S3 = 0;
    S4 = 0;
    S5 = 0;
    S6 = 0;
    k1 = 1;
    k2 = 2;
    k3 = 3;
    k4 = 4;
    k5 = 5;
"""

Next we actually fit the model to some data. This just requies two lines:

1. Create the fitting object: This requires the model (antimony or roadrunner), the data to fit the model and what parameters you want to fit.
2. The second line causes the fitting to take place.

The data file is just a normal CSV file, where the first column is time and the remaining columns are what ever you have measured.
In this case the data file has all five-step species recorded in time, hence it has one column for utime and six columns for the siz species.

By default, the fitter will use all the data it finds in the data file unless specified otherwise.


In [3]:
# Fit parameters to ts1
fitter = SBstoat.ModelFitter(ANTIMONY_MODEL, "tst_data.txt", ["k1", "k2", "k3", "k4", "k5"])
fitter.fitModel()

If the fit was successful (no error is reported),
you can ask the fitter for information on what the results were using the reportFit call:


In [4]:
# Provide fit details
print(fitter.reportFit())

[[Variables]]
    k1:  0.955789442860191
    k2:  2.2407776807094004
    k3:  2.967638483699222
    k4:  3.0765412667273306
    k5:  5.908006830802661
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 44
    # data points      = 180
    # variables        = 5
    chi-square         = 73.2546170
    reduced chi-square = 0.41859781
    Akaike info crit   = -151.822803
    Bayesian info crit = -135.858019
[[Correlations]] (unreported correlations are < 0.100)
    C(k4, k5) = -0.248
    C(k3, k4) = -0.226
    C(k2, k3) = -0.218
    C(k3, k5) = -0.211
    C(k2, k4) = -0.189
    C(k1, k2) = -0.179
    C(k2, k5) = -0.178
    C(k1, k3) = -0.147
    C(k1, k5) = -0.144
    C(k1, k4) = -0.141


More advanced users may want control over the optimizer method used to do the fit as well as the parameters passed to these methods. Below is an example for the ``differential_evolution`` method.

In [20]:
optimizerMethod = SBstoat.ModelFitter.OptimizerMethod(
    method="differential_evolution",
    kwargs={ "popsize": 10000, "atol": 0.001, "maxiter": 1000})
fitter = SBstoat.ModelFitter(ANTIMONY_MODEL, "tst_data.txt", ["k1", "k2", "k3", "k4", "k5"], fitterMethods=[optimizerMethod])
fitter.fitModel()
print(fitter.reportFit())

[[Variables]]
    k1:  0.73379992896702
    k2:  2.6536557048047187
    k3:  4.469159188466452
    k4:  9.75697867697164
    k5:  4.441457750957991
[[Fit Statistics]]
    # fitting method   = differential_evolution
    # function evals   = 101
    # data points      = 180
    # variables        = 5
    chi-square         = 768.708742
    reduced chi-square = 4.39262138
    Akaike info crit   = 271.315954
    Bayesian info crit = 287.280738
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.


In [ ]:
# Plot the fitl
fitter.plotFitAll(numRow=2, numCol=3)

In [ ]:
# Plot residuals
fitter.plotResiduals(numRow=2, numCol=3)

In [ ]:
# Many residuals plots are available.
fitter.plotResidualsAll(numCol=3, titleFontsize=8)

In [ ]:
# Get estimates of parameter variance by using bootstrapping
fitter.bootstrap(numIteration=2000, reportInterval=500)
print(fitter.getParameterStds())

In [ ]:
# Look at the distribution of the parameter fits.
fitter.plotParameterHistograms(figsize=(10,8), ylim=[0, 3], xlim=[0, 6], bins=50, titlePosition=[.5,1])

In [ ]:
?fitter.plotFitAll

In [ ]:
# Look at interactions between parameter estimates
fitter.plotParameterEstimatePairs(figsize=(10,8), ylim=[0, 10], xlim=[0, 10], markersize=2)

In [ ]:
fitter.reportBootstrap()

## Detailed Tutorial

In [ ]:
# Import the modules needed.
import numpy as np
TIME = SBstoat.TIME

## Manipulating Tabular Data
Tables are a great way to describe data. Tables are a collection of values organized into rows and columns. Typically, the columns have names. Common operations on tables are:

- Create a table from a file
- Create a new table from a subset of rows and/or columns
- Create new columns and/rows by performing addition, subtaction, and other operations on table rows and columns.

Many systems provide table representations. Spreadsheets are widely used. In the R language, a dataframe is a table. In python, the pandas package provides a python version of dataframes.

Pandas is very powerful, but it is slow. numpy provides arrays and matrices, but these lack the convenience of column names. In addition, the variables we analyze are collected at the same timepoints, and so it's important to associate these timepoints with the variable values.

``NamedTimeseries`` is a thin layer on top of numpy ``ndarray`` that provides an efficient table abstraction. Efficiency is a particular concern for model fitting since fitting can be very numerically intensive.

Let's start by creating a ``NamedTimeseries`` from the file ``tst_data.txt``. In python, an object is created from a constructor that is the class name followed by arguments. There are several constructors for ``NamedTimeseries``.

In [ ]:
ts1 = SBstoat.NamedTimeseries(csvPath="tst_data.txt")  # Construct from a file
print(ts1)

There are 7 columns (including "time") and 30 rows. All ``NamedTimeseries`` have a "time" column. In fact, the python variable ``TIME`` can be used to reference this column.

We reference rows by their "index", their count from 0. In fact, we can make a new ``NamedTimeseries`` that consists of a subset of the rows.

In [ ]:
# Get the first row
print(ts1[1])

In [ ]:
# Get rows 1, 3, 5
print(ts1[[1, 3, 5]])

In [ ]:
# Get rows 1 through 5
# This is a slice; you specify the first row you want and where the slice stops
ts2 = ts1[1:6]
print(ts2)

We can also select columns. A column select produces a ``numpy`` ``ndarray``, not at ``NamedArray``. This is done so that we can manipulate column selections using the powerful numpy facilities.

In [ ]:
ts2["S1"]  # Column S1 in ts2

To get the numpy array for a list of columns, we give the list.

In [ ]:
ts2[["S1", "S2"]]

To get a **new timeseries** that just has "S1", you use ``subsetColumns``.

In [ ]:
ts2a = ts2.subsetColumns("S1")
print(ts2a)

In [ ]:
# Create a timeseries from ts2 with just S1, S2
print(ts2.subsetColumns(["S1", "S2"]))

We can add columns to the timeseries. Here, we add a column with a constant value of 1.

In [ ]:
ts2["constant_1"] = 1
print(ts2)

We can also do calculations and create new columns with the results.

In [ ]:
ts2["New_Calculation"] = ts2["S1"] + np.multiply(ts2["S2"], ts2[TIME])
ts2["Other_Calculation"] = ts2["S1"] + 10*np.multiply(ts2["S2"], ts2[TIME])
print(ts2)

In [ ]:
# Let's get rid of Other_Calculation
del ts2["Other_Calculation"]
print(ts2)

We can combine the rows and columns of a ``NamedTimeseries`` to make a new ``NamedTimeseries``.

In [ ]:
ts3 = ts2.subsetColumns(["S1", "S2"])
ts4 = ts2.subsetColumns("S3")
ts5 = ts2.subsetColumns("New_Calculation")
ts6 = ts3.concatenateColumns([ts4, ts5])  # Has S1, S2, S3, New_Calculation"
print(ts6)

In [ ]:
ts7 = ts1[:2]
ts8 = ts1[2:4]
ts9 = ts1[4:6]
ts10 = ts7.concatenateRows([ts8, ts9])  # First 6 rows of ts1
print(ts10)

In [ ]:
# Save the result in a file
ts2.to_csv("new_data.csv")

In [ ]:
# To see the file, we run a command in the operating system shell. The following works in Unix and Mac.
!cat new_data.csv

## Plotting Tabular Data
We provide 4 types of plots with user controllable options for the plots. The plots are:

1. Plot all variables of one table on a single plot.
1. Plot all variables of one table in one plot and all variables of second plot
1. Plot each variable of a table in a separate plot
1.  Plot each variable of two tables in the same plot

In [ ]:
# Create the plotter object
plotter = SBstoat.TimeseriesPlotter()

In [ ]:
# To see the plotting options...
?plotter.plotHistograms

In [ ]:
# Case 1: Plot all variables of a single table on a single plot
plotter.plotTimeMultiple(ts1)
plotter.plotTimeMultiple(ts1, timeseries2=ts1, marker="o")

In [ ]:
# We can also plot a subset of the variables in a single plot
plotter.plotTimeMultiple(ts1, columns=["S1", "S6"])

In [ ]:
# Add to the plot with options
plotter.plotTimeMultiple(ts1, columns=["S1", "S6"], ylabel="concetration", title="Two Variables")

In [ ]:
# Create a second table for comparisons. We can access all non-time variables using the property colnames.
ts4 = ts1.copy()  # Create a copy of the timeseries
ts4[ts4.colnames] = 5*np.cos(ts4[ts4.colnames]) + 2*ts4[ts4.colnames]

In [ ]:
# Case 2: Plot two multiple variable plots, side-by-side
plotter.plotTimeMultiple(ts1, timeseries2=ts4)

In [ ]:
# Case 3: Separate plot for each variable of a single table
plotter.plotTimeSingle(ts1, numRow=2, numCol=3, timeseries2=ts2,
                      figsize=(10,8), marker="o")

In [ ]:
# Case 4: Separate plot for each variable for two tables
plotter.plotTimeSingle(ts1, timeseries2=ts2, figsize=(10, 8))

## Fitting Parameters of a Model
Kinetics models frequently have unknown constants that need to be estimated. This process of estimation is called model fitting, or just fitting. Fitting involves running multiple simulations with different values of parameters to find values that result in a good match between simulated and measured species concentrations.

In [ ]:
ANTIMONY_MODEL = """ 
# Reactions   
    J1: S1 -> S2; k1*S1
    J2: S2 -> S3; k2*S2
    J3: S3 -> S4; k3*S3
    J4: S4 -> S5; k4*S4
    J5: S5 -> S6; k5*S5;
# Species initializations
    S1 = 10;
    S2 = 0;
    S3 = 0;
    S4 = 0;
    S5 = 0;
    S6 = 0;
    k1 = 1;
    k2 = 2;
    k3 = 3;
    k4 = 4;
    k5 = 5;
"""

This model has the parameters ``k1, k2, k3, k4, k5``, the kinetics constants for their respective reactions. In this case, we know the true value of the parameters, which are specified at the bottom of the model. This knowledge provides a way to explore the effectiveness of model fitting.

This model is a linear pathway where mass moves from species ``S1`` to ``S6`` over time. The rate at which this happens depends on the kinetics constants.

Let's see how well we can fit this model to the ``ts1`` data described in the first section.

In [ ]:
print(ts1)

The fitting data has values for each of the six floating species. Model fitting works iteratively varying the values of the parameters, simulating the model with these parameters, and then evaluating how close the simulation is to the fitting data.

So, to do a fit, we must specify:

1. The model to fit
1. The fitting data
1. The parameters to estimate

To elaborate on point (3), often models contain some parameters whose values are known and others that aren't.

The class ``ModelFitter`` is used to do fitting. The constructor must specify all 3 items above. If parameters is None, then no fitting is done.

In [ ]:
# Run the model using the parameter values in the model. There is no fitting.
base_fitter = SBstoat.ModelFitter(ANTIMONY_MODEL, ts1, None)

There are other options as well, such as choosing a subset of columns for the fit.

In [ ]:
# Fit parameters to ts1
fitter = SBstoat.ModelFitter(ANTIMONY_MODEL, "tst_data.txt", ["k1", "k2", "k3", "k4", "k5"], selectedColumns=["S1", "S2"])

In [ ]:
# Fit parameters to ts1 S1, S2
fitter = SBstoat.ModelFitter(ANTIMONY_MODEL, "tst_data.txt", ["k1", "k2", "k3", "k4", "k5"])
fitter.fitModel()

To fit a model, we use the ``fitModel`` method. We can then see the optimized residual variance and the values of the model parameters.

In [ ]:
base_fitter.fitModel()

In [ ]:
# The model without fitting any parameters
ts_base = SBstoat.NamedTimeseries(namedArray=base_fitter.roadrunnerModel.simulate(0, 4, 30))
plotter.plotTimeSingle(ts_base, numRow=2)

This is a bit crowded for detailed comparisons. So, we make comparisons between each variable.

In our running example, the values of parameters in the original model are their true value. Ideally, we want to achieve a variance close to this. We create a new fitter object since we still want to have access to the unchanged model that we started with.

In [ ]:
parameterNames = ["k%d" % i for i in range(1, 6)]
fitter = SBstoat.ModelFitter(ANTIMONY_MODEL, ts1, parameterNames)
fitter.fitModel()
print("residual variance of the fitted model: %3.2f" % np.var(fitter.residualsTS.flatten()))
print("Values of fitted parameters: %s" % str(fitter.getParameterMeans()))

Now let's compare the fitted model with the data.

In [ ]:
fittedModel = fitter.roadrunnerModel
fittedModel.reset()
fittedTS = SBstoat.NamedTimeseries(namedArray=fittedModel.simulate(ts1.start, ts1.end, len(ts1)))
plotter.plotTimeMultiple(ts1, timeseries2=fittedTS, marker="o")

In [ ]:
plotter.plotTimeSingle(ts1, timeseries2=fittedTS, numRow=2, numCol=3, figsize=(12, 10), marker="o")

In [ ]:
# Plot pairs of values
plotter.plotValuePairs(fittedTS, pairs=[("S1", "S2"), ("S1", "S6"), ("S2", "S3")], numCol=2)

In [ ]:
fitter.plotFitAll(numCol=2, numRow=1, isMultiple=True)

In [ ]:
fitter.plotFitAll(numRow=2, numCol=2, figsize=(12, 10), color=["red", "blue"], alpha=[1.0, 0.3],
                 markersize=3)

Sometimes, the observed values are in different units than what is produced by the simulation. So, fitted values must be transformed.
This is done by specifying fittedDataTransformationDct, a dictionary of functions for transformming fitted values.

In [ ]:
# Specify a data transformation for S1. It should be a function of the entire timeseries and return a new value for S1.
FACTOR = 10
def transformS1(ts):
    return FACTOR*ts["S1"]

In [ ]:
# Create transformed observed data
ts1_new = ts1.copy()
ts1_new["S1"] = transformS1(ts1)

Note that running the fit where we have transformed the observational data and the fitted data by the same amount results in substantially the same fit.

In [ ]:
fitter = SBstoat.ModelFitter(ANTIMONY_MODEL, ts1_new, parameterNames, fittedDataTransformDct={"S1": transformS1})
fitter.fitModel()
print(fitter.reportFit())